# 특성맵(feature map) 시각화

Xception 모델을 불러오겠습니다.  
파라미터들은 ImageNet으로 이미 훈련되어 있습니다.

In [ ]:
from tensorflow import keras

model = keras.applications.xception.Xception()
model.summary()

고양이 이미지를 다운받고 출력해보겠습니다.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as image

img_path = keras.utils.get_file(
    fname="cat.jpg",
    origin="https://img-datasets.s3.amazonaws.com/cat.jpg")
img = image.imread(img_path)
plt.imshow(img)
plt.axis('off')
plt.show()
print(img.shape)

Xception의 입력해상도 299$\times$299로 변환하고 이걸 다시 numpy array형으로 변환합니다.

In [ ]:
import tensorflow as tf
import numpy as np

img = tf.image.resize(img,(299,299))
img = keras.utils.img_to_array(img)

plt.imshow(img/255)
plt.axis("off")
plt.show()

Xception이 예측한 5가지 클래스가 모두 고양이 품종입니다.

In [ ]:
from tensorflow import keras
import gdown, zipfile, os

if not os.path.isfile('VGG16_test.zip'):
    gdown.download(id='1K46V-TnwB8EWuk0RGE-oloVs9HfHRbGp', output='imagenet_classes.txt')

with open('imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]

img = tf.expand_dims(img,axis=0)
out=model(tf.keras.applications.xception.preprocess_input(img))

top_5 = np.argsort(-out[0])[:5]
for idx in top_5:
    print(f"{labels[idx]} : {out[0,idx]*100:.4f}%")

`isinstance`는 layer가 클래스 `Conv2D` 또는 클래스 `SeparableConv2D`의 인스턴스면 `True`를 리턴하고 그렇지 않으면 `False`를 리턴합니다.  
합성곱층에서 출력되는 특성맵들을 리스트로 묶습니다.  
함수형 API로 이미지가 입력되면 특성맵(feature map)들의 리스트가 출력되는 다중 출력 모델을 만듭니다.

In [ ]:
from tensorflow.keras.layers import Conv2D, SeparableConv2D

layer_outputs = []
layer_names = []
for layer in model.layers:
    if isinstance(layer, (Conv2D, SeparableConv2D)):
        layer_outputs.append(layer.output)
        layer_names.append(layer.name)
activation_model = keras.Model(inputs=model.input, outputs=layer_outputs)

[tf.keras.applications.xception.preprocess_input](https://www.tensorflow.org/api_docs/python/tf/keras/applications/xception/preprocess_input)는 입력 데이터를 전처리한 후 덮어쓰기 때문에 다시 적용하면 안됩니다.  
합성곱층에서 출력되는 특성맵들의 해상도와 채널 깊이수입니다.  
max pooling층을 통과할때마다 해상도가 절반씩 줄어듭니다.  
합성곱층 필터가 2배 증가할때는 채널의 깊이도 2배 늘어납니다.

In [ ]:
activations = activation_model.predict(img)

for i in range(len(activations)):
    print(activations[i].shape)

2번째 합성곱층이 출력하는 피처맵 64장중 앞 25장입니다.
- (1,1)번째 필터는 경계선에 반응하는 듯합니다.
- (1,2)번째 필터는 고양이의 흰 색깔에 반응하는 듯합니다.  
- (1,3)번째 필터는 세로 경계선에 반응하는 듯합니다.  
- (2,2)번째 필터는 가로 경계선에 반응하는 듯합니다.  
- (5,4)번째 필터는 고양이의 파란 눈에 반응하는 듯합니다.  

In [ ]:
n=1

print(activations[n][0,:,:,:].shape)

plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(activations[n][0,:,:,i], cmap='gray')
    plt.xticks([])
    plt.yticks([])
plt.show()

8번째 합성곱층이 출력하는 피처맵 256장중 앞 25장입니다.  
max pooling층을 두번 거치며 해상도가 1/4로 줄었습니다.

In [ ]:
n=7

print(activations[n][0,:,:,:].shape)

plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(activations[n][0,:,:,i], cmap='gray')
    plt.xticks([])
    plt.yticks([])
plt.show()

11번째 합성곱층이 출력하는 피처맵 728장중 앞 25장입니다.  
max pooling층을 거치며 해상도가 절반으로 줄었습니다.  
이제는 이미지 소스가 고양이였다는걸 알아보기 힘듭니다.

In [ ]:
n=10

print(activations[n][0,:,:,:].shape)

plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(activations[n][0,:,:,i], cmap='gray')
    plt.xticks([])
    plt.yticks([])
plt.show()

마지막 합성곱층이 출력하는 피처맵 2048장중 앞 25장입니다.  
max pooling층을 거치며 해상도가 절반으로 줄었습니다.  
이제는 전혀 알아볼수가 없습니다.

In [ ]:
n=-1

print(activations[n][0,:,:,:].shape)

plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(activations[n][0,:,:,i], cmap='gray')
    plt.xticks([])
    plt.yticks([])
plt.show()

# 필터 시각화

첫번째 합성곱층에는 32개의 3$\times$3$\times$3 필터가 있습니다.  
앞 25장의 첫번째 채널을 시각화한 이미지들입니다.  
무슨 의미인지 모르겠네요.  
사실 다른 층의 필터들도 출력해보면 이해할수 없긴 마찬가지입니다.  
이번 섹션의 목표는 필터마다 이해가능한 이미지를 대응시키는 것입니다.

In [ ]:
layer = model.get_layer(name="block1_conv1")

print(layer.weights[0].shape)

plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(np.array(layer.weights[0])[:,:,0,i], cmap='gray')
    plt.xticks([])
    plt.yticks([])
plt.show()

입력해상도를 299$\times$299에서 200$\times$200으로 바꾸기 위해 Xception을 다시 불러오겠습니다.  
`include_top=False`로 설정하면 입력해상도를 변경할 수 있습니다.  
299$\times$299로도 이번 섹션의 목적을 달성할 수 있지만 출력해보면 200$\times$200이 더 그럴듯해 보입니다.  
이미지가 입력되면 특성맵들의 리스트가 출력되는 다중 출력 모델을 다시 만듭니다.

In [ ]:
model = keras.applications.xception.Xception(
    weights="imagenet",
    include_top=False)

layer_outputs = []
layer_names = []
for layer in model.layers:
    if isinstance(layer, (Conv2D, SeparableConv2D)):
        layer_outputs.append(layer.output)
        layer_names.append(layer.name)
activation_model = keras.Model(inputs=model.input, outputs=layer_outputs)

필터마다 어떤 이미지를 대응시키는 함수 `visualize_filter`를 정의하겠습니다.  
`visualize_filter`를 블럭별로 해석하면 다음과 같습니다.
- 200$\times$200 해상도의 컬러 이미지를 랜덤하게 만듭니다. 픽셀값은 0.4~0.6사이에서 균등분포를 따릅니다.
- 이미지를 입력했을 때 `conv_index`번째 합성곱층에서 출력하는 특성맵들을 생각합니다. 그중에서 `filter_index`번째 필터에 대응하는 특성맵을 뽑아내고 주변 2픽셀을 삭제한후 모든 픽셀값들을 평균하겠습니다. 함수 $F$는 이 대응관계로 정의되는 함수입니다. 자동미분을 하기위해 `GradientTape`안에서 정의하고 image는 텐서플로우 변수형으로 바꿔줍니다.
- 함수 $F$를 입력 이미지로 미분해서 그레디언트를 구합니다. 그리고 크기 1로 노멀라이즈 합니다. 학습률 10으로 30번 경사상승법을 적용합니다. 함수 $F$의 최대점을 찾아가는데 목적입니다. 
- 텐터플로우 변수형은 수정이 안되기 때문에 일단 numpy array형으로 바꿉니다. 평균 0, 표준편차 1로 노멀라이즈한 후 다시 평균 128, 표준편차 64로 바꿔줍니다. 0과 255를 벗어나는 픽셀은 모두 0으로 클리핑한 후 정수형으로 바꿔줍니다. 가장자리 25픽셀들은 삭제합니다.

함수 $F$의 최대점은 `conv_index`번째 합성곱층의 `filter_index`번째 필터에 가장 적극적으로 반응하는 이미지라고 해석할 수 있습니다.

In [ ]:
import tensorflow as tf

def visualize_filter(conv_index, filter_index):
    img = tf.random.uniform(minval=0.4, maxval=0.6, shape=(1, 200, 200, 3))
    keras.applications.xception.preprocess_input(img)
    
    for i in range(30):
        with tf.GradientTape() as tape:
            tape.watch(img)
            activations = activation_model(img)
            activation = activations[conv_index]
            F = tf.reduce_mean(activation[:, 2:-2, 2:-2, filter_index])

        grads = tape.gradient(F, img)
        grads = tf.math.l2_normalize(grads)
        img += 10 * grads
    
    img = img[0].numpy()
    img -= img.mean()
    img /= img.std()
    img *= 64
    img += 128
    img = np.clip(img, 0, 255).astype("uint8")
    img = img[25:-25, 25:-25, :]
    
    return img

첫번째 합성곱층의 32개 필터중 앞 25장에 가장 적극적으로 반응하는 입력 이미지들입니다.  
색깔 또는 매우 단순한 기하적 패턴에 반응합니다.

In [ ]:
n=0

print(activations[n].shape[-1])

plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(visualize_filter(n,i))
    plt.xticks([])
    plt.yticks([])
plt.show()

세번째 합성곱층의 128개 필터중 앞 25장에 가장 적극적으로 반응하는 입력 이미지들입니다.  
세로선, 가로선, 사선에 반응하는 듯 보입니다.

In [ ]:
n=2

print(activations[n].shape[-1])

plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(visualize_filter(n,i))
    plt.xticks([])
    plt.yticks([])
plt.show()

9번째 합성곱층의 728개 필터중 앞 25장에 가장 적극적으로 반응하는 입력 이미지들입니다.  
복잡한 패턴들에 반응하기 시작합니다.

In [ ]:
n=8

print(activations[n].shape[-1])

plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(visualize_filter(n,i))
    plt.xticks([])
    plt.yticks([])
plt.show()

14번째 합성곱층의 728개 필터중 앞 25장에 가장 적극적으로 반응하는 입력 이미지들입니다.

In [ ]:
n=13

print(activations[n].shape[-1])

plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(visualize_filter(n,i))
    plt.xticks([])
    plt.yticks([])
plt.show()

21번째 합성곱층의 728개 필터중 앞 25장에 가장 적극적으로 반응하는 입력 이미지들입니다.

In [ ]:
n=20

print(activations[n].shape[-1])

plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(visualize_filter(n,i))
    plt.xticks([])
    plt.yticks([])
plt.show()